# Ensemble Learning

### Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

### Read the CSV and Perform Basic Data Cleaning

In [2]:
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
df=pd.get_dummies(df,columns=["home_ownership","verification_status","application_type","hardship_flag","debt_settlement_flag","issue_d","pymnt_plan","initial_list_status","next_pymnt_d"],drop_first=True)
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,application_type_Joint App,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,0,1,1,0,0,0,1,1,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,0,0,0,1,0,0,1,1,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,0,0,0,1,0,0,1,1,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,0,1,0,1,0,0,1,1,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,0,0,0,0,0,0,1,1,1


###  Split the Data into Training and Testing

In [3]:
X = df.copy()
X.drop("loan_status", axis=1,inplace=True)
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,application_type_Joint App,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.526309,0.106747,0.357659,0.373992,0.148001,0.139660,0.451066,0.177238,0.876121,0.616839
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.499311,0.308793,0.479314,0.483865,0.355104,0.346637,0.497603,0.381873,0.329446,0.486161
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
y=df["loan_status"]
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

###  Data Pre-Processing

In [10]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_scaled = scaler.fit(X_train)

In [11]:
X_scaled_train=X_scaled.transform(X_train)
X_scaled_test=X_scaled.transform(X_test)

##  Balanced Random Forest Classifier

In [12]:
from imblearn.ensemble import BalancedRandomForestClassifier

In [13]:
brf=BalancedRandomForestClassifier(n_estimators=100,random_state=1)
brf_model= brf.fit(X_train,y_train)

### Calculated the balanced accuracy score

In [14]:
y_pred_brf=brf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_brf)

0.7992101788919226

### Display the confusion matrix

In [15]:
confusion_matrix(y_test, y_pred_brf)

array([[   73,    31],
       [ 1770, 15331]])

### Print the imbalanced classification report

In [16]:
print(classification_report_imbalanced(y_test, y_pred_brf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.70      0.90      0.07      0.79      0.62       104
   low_risk       1.00      0.90      0.70      0.94      0.79      0.64     17101

avg / total       0.99      0.90      0.70      0.94      0.79      0.64     17205



### List the features sorted in descending order by feature importance

In [17]:
sorted(zip(brf_model.feature_importances_,X.columns),reverse=True)

[(0.07893662638888792, 'total_rec_prncp'),
 (0.06957418932100731, 'total_pymnt'),
 (0.0646001570842341, 'total_rec_int'),
 (0.05722147278414711, 'last_pymnt_amnt'),
 (0.05534202633157051, 'total_pymnt_inv'),
 (0.026165015739022367, 'int_rate'),
 (0.018505472650741542, 'tot_hi_cred_lim'),
 (0.018086014502427892, 'installment'),
 (0.017690728601837142, 'tot_cur_bal'),
 (0.017391901261611917, 'issue_d_Jan-2019'),
 (0.01684161926700493, 'max_bal_bc'),
 (0.01658483826983536, 'issue_d_Mar-2019'),
 (0.015685085200333706, 'il_util'),
 (0.015287919374801876, 'mo_sin_old_rev_tl_op'),
 (0.015187633959384045, 'out_prncp_inv'),
 (0.015184968503838882, 'total_bc_limit'),
 (0.01517745443751965, 'total_bal_il'),
 (0.014789943339863379, 'mo_sin_old_il_acct'),
 (0.014779345574474662, 'bc_util'),
 (0.014610214881872143, 'total_rev_hi_lim'),
 (0.014604065747598257, 'all_util'),
 (0.014377880279974936, 'out_prncp'),
 (0.014006889432055077, 'revol_bal'),
 (0.013847073935733006, 'dti'),
 (0.01336363796898319

### Easy Ensemble Classifier

In [18]:
from imblearn.ensemble import EasyEnsembleClassifier
easy= EasyEnsembleClassifier(n_estimators=100,random_state=42)
easy=easy.fit(X_train,y_train)

### Calculated the balanced accuracy score

In [20]:
y_pred_easy = easy.predict(X_test)
balanced_accuracy_score(y_test, y_pred_easy)

0.9289546157894499

### Display the confusion matrix

In [21]:
confusion_matrix(y_test, y_pred_easy)

array([[   95,     9],
       [  950, 16151]])

### Print the imbalanced classification report

In [22]:
print(classification_report_imbalanced(y_test, y_pred_easy))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.91      0.94      0.17      0.93      0.86       104
   low_risk       1.00      0.94      0.91      0.97      0.93      0.87     17101

avg / total       0.99      0.94      0.91      0.97      0.93      0.87     17205



##  Final Questions

- Which model had the best balanced accuracy score?

Easy Ensemble Classifier

- Which model had the best recall score?

Easy Ensemble Classifier

- Which model had the best geometric mean score?

Easy Ensemble Classifier

- What are the top three features?

- total_rec_prncp
- total_pymnt
- total_rec_int